# Load Data

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
if 'google.colab' in str(get_ipython()):
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/dataset/DSA4212/assignment_2/

Mounted at /content/drive
/content/drive/MyDrive/dataset/DSA4212/assignment_2


In [3]:
pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.1-cp39-cp39-linux_x86_64.whl size=582303 sha256=b81fbc50e555a39c4606c5d8aa8b899977b4607894a4b63ee8478a9443a280d4
  Stored in directory: /root/.cache/pip/wheels/5b/7d/31/9a9a4993d085bc85bee21946bce94cd5906ce99730f5467e57
Successfully built annoy


In [4]:
import matplotlib.pyplot as plt
import numpy as onp
import pandas as pd
import re
from scipy.stats import norm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from annoy import AnnoyIndex

In [5]:
anime = pd.read_csv("assignment_2_anime.csv")
train = pd.read_csv("assignment_2_ratings_train.csv")
test = pd.read_csv("assignment_2_ratings_test.csv")

In [6]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [7]:
len(anime)

12294

In [8]:
len(anime.anime_id.unique())

12294

In [9]:
train.head()

,user_id,anime_id,rating
0,20170,10794,6
1,24592,21995,5
2,18358,7054,10
3,59267,488,7
4,69313,30544,4


In [10]:
len(train)

4436068

In [11]:
test.head()

,user_id,anime_id,rating
0,44017,13161,4
1,14307,14993,7
2,55155,268,9
3,63515,2889,9
4,54059,2581,7


In [12]:
len(test)

1901173

In [13]:
avg_user_train_rating = train.groupby("user_id", as_index = False).agg({"rating": [onp.mean]})

# Content-Based 100 neighbours & simple average (1.85)

In [ ]:
anime2 = anime.copy()

In [ ]:
anime2.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
anime2["text"] = anime2["name"] + " " + anime2["genre"]

In [ ]:
anime2["text"] = anime2["text"].astype(str)

In [ ]:
def remove_numbers_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: ''.join([i for i in text if not i.isdigit()])
        )
    return df

def remove_punctuations_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: re.sub(r'[^\w\s]', ' ', text)
        )
    df[dst_col] = df[src_col].apply(
        lambda text: text.replace("_", " ")
        )
    return df

def replace_multiple_spaces_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: re.sub(' +', ' ', text)
    )
    return df

def lowercase_string_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: text.lower()
        )
    return df

In [ ]:
anime2 = anime2.pipe(remove_numbers_df, src_col = "text", dst_col = "text").\
    pipe(remove_punctuations_df, src_col = "text", dst_col = "text").\
    pipe(replace_multiple_spaces_df, src_col = "text", dst_col = "text").\
    pipe(lowercase_string_df, src_col = "text", dst_col = "text")

In [ ]:
anime2.drop(["name", "genre"], axis = 1, inplace = True)

In [ ]:
anime2

,anime_id,type,episodes,rating,members,text
0,32281,Movie,1,9.37,200630,kimi no na wa drama romance school supernatural
1,5114,TV,64,9.26,793665,fullmetal alchemist brotherhood action adventu...
2,28977,TV,51,9.25,114262,gintama action comedy historical parody samura...
3,9253,TV,24,9.17,673572,steins gate sci fi thriller
4,9969,TV,51,9.16,151266,gintama action comedy historical parody samura...
...,...,...,...,...,...,...
12289,9316,OVA,1,4.15,211,toushindai my lover minami tai mecha minami he...
12290,5543,OVA,1,4.28,183,under world hentai
12291,5621,OVA,4,4.88,219,violence gekiga david no hoshi hentai
12292,6133,OVA,1,4.98,175,violence gekiga shin david no hoshi inma dense...


In [ ]:
vectorizer = TfidfVectorizer()
tfidf_text_vectors = vectorizer.fit_transform(anime2.text)

In [ ]:
tfidf_text_df = pd.DataFrame(tfidf_text_vectors.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
anime2.drop(["text"], axis = 1, inplace = True)

In [ ]:
anime2

,anime_id,type,episodes,rating,members
0,32281,Movie,1,9.37,200630
1,5114,TV,64,9.26,793665
2,28977,TV,51,9.25,114262
3,9253,TV,24,9.17,673572
4,9969,TV,51,9.16,151266
...,...,...,...,...,...
12289,9316,OVA,1,4.15,211
12290,5543,OVA,1,4.28,183
12291,5621,OVA,4,4.88,219
12292,6133,OVA,1,4.98,175


In [ ]:
mean_episodes = anime2[anime2.episodes != "Unknown"].episodes.astype(int).mean()

In [ ]:
anime2["episodes"] = anime2["episodes"].apply(lambda x: int(x) if x != "Unknown" else mean_episodes)

In [ ]:
scaler = MinMaxScaler()
scaler_vectors = scaler.fit_transform(anime2[["episodes", "rating", "members"]])

In [ ]:
scaler_df = pd.DataFrame(scaler_vectors, columns=scaler.get_feature_names_out())

In [ ]:
scaler_df

,episodes,rating,members
0,0.000000,0.924370,0.197872
1,0.034673,0.911164,0.782770
2,0.027518,0.909964,0.112689
3,0.012658,0.900360,0.664325
4,0.027518,0.899160,0.149186
...,...,...,...
12289,0.000000,0.297719,0.000203
12290,0.000000,0.313325,0.000176
12291,0.001651,0.385354,0.000211
12292,0.000000,0.397359,0.000168


In [ ]:
anime2 = pd.get_dummies(anime2)

In [ ]:
anime2

,anime_id,episodes,rating,members,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,32281,1.0,9.37,200630,1,0,0,0,0,0
1,5114,64.0,9.26,793665,0,0,0,0,0,1
2,28977,51.0,9.25,114262,0,0,0,0,0,1
3,9253,24.0,9.17,673572,0,0,0,0,0,1
4,9969,51.0,9.16,151266,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
12289,9316,1.0,4.15,211,0,0,0,1,0,0
12290,5543,1.0,4.28,183,0,0,0,1,0,0
12291,5621,4.0,4.88,219,0,0,0,1,0,0
12292,6133,1.0,4.98,175,0,0,0,1,0,0


In [ ]:
anime2.drop(["episodes", "rating", "members"], axis = 1, inplace = True)

In [ ]:
anime2 = pd.concat([anime2, scaler_df, tfidf_text_df], axis = 1)

In [ ]:
anime2.shape

(12294, 11862)

In [ ]:
anime2.rating = anime2.rating.fillna(anime2.rating.mean())

In [ ]:
anime2

,anime_id,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,episodes,rating,members,...,zushiki,zushioumaru,zusshiri,zutto,zvezda,zwei,zz,étude,üks,ēldlive
0,32281,1,0,0,0,0,0,0.000000,0.924370,0.197872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,0,0,0,0,0,1,0.034673,0.911164,0.782770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977,0,0,0,0,0,1,0.027518,0.909964,0.112689,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,0,0,0,0,0,1,0.012658,0.900360,0.664325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969,0,0,0,0,0,1,0.027518,0.899160,0.149186,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,9316,0,0,0,1,0,0,0.000000,0.297719,0.000203,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12290,5543,0,0,0,1,0,0,0.000000,0.313325,0.000176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12291,5621,0,0,0,1,0,0,0.001651,0.385354,0.000211,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12292,6133,0,0,0,1,0,0,0.000000,0.397359,0.000168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
nearest_neighbours = AnnoyIndex(anime2.shape[1] - 1, 'angular')
for index, row in anime2.drop(["anime_id"], axis = 1).iterrows():
    if index % 5000 == 0:
        print(index)
    nearest_neighbours.add_item(index, row)

nearest_neighbours.build(100)
# 4 mins

0
5000
10000


True

In [ ]:
neighbours = {}
num_neighbours = 100
for i in range(len(anime2)):
    if i % 5000 == 0:
        print(i)
    ith_neighbours = nearest_neighbours.get_nns_by_item(i, num_neighbours)
    ith_id = anime2.loc[i, "anime_id"]

    neighbours[ith_id] = list(map(lambda x: anime.loc[x, "anime_id"], ith_neighbours))

# 7 mins

0
5000
10000


In [ ]:
train_dict = {}
for index, row in train.iterrows():
    temp_dict = train_dict.get(int(row["user_id"]), {})
    temp_dict[int(row["anime_id"])] = int(row["rating"])
    train_dict[int(row["user_id"])] = temp_dict

In [ ]:
predicted_test_8 = test.copy()
predicted_test_8.loc[:, "predicted_rating"] = -1

for index, row in predicted_test_8.iterrows():
    user_id = row["user_id"]
    anime_id = row["anime_id"]

    if anime_id not in neighbours:
        continue
    anime_id_neighbours = neighbours[anime_id]

    if user_id not in train_dict:
        continue
    user_ratings_train = train_dict[user_id]

    if index % 50000 == 0:
        print(index)

    temp = []
    for neighbour in anime_id_neighbours:
        if neighbour in user_ratings_train:
            temp.append(user_ratings_train[neighbour])

    predicted_test_8.loc[index, "predicted_rating"] = onp.mean(temp)

# 6 mins

0


/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000


In [ ]:
predicted_test_8[predicted_test_8["predicted_rating"] == -1]

,user_id,anime_id,rating,predicted_rating
896,69461,269,10,-1.0
2077,70751,4722,9,-1.0
2306,1360,16498,10,-1.0
5241,45465,918,10,-1.0
6261,60842,853,9,-1.0
...,...,...,...,...
1897212,35,9001,10,-1.0
1897656,39943,1562,9,-1.0
1898495,2512,1535,9,-1.0
1900049,42660,9760,7,-1.0


In [ ]:
predicted_test_8[predicted_test_8["predicted_rating"].isna()]

,user_id,anime_id,rating,predicted_rating
11,41101,304,10,NaN
71,69875,27655,10,NaN
73,2063,18393,10,NaN
86,44216,12445,9,NaN
133,11269,6707,9,NaN
...,...,...,...,...
1901043,9133,1078,6,NaN
1901115,48526,15037,5,NaN
1901121,73420,901,3,NaN
1901130,62099,11371,9,NaN


In [ ]:
predicted_test_9 = predicted_test_8[(predicted_test_8["predicted_rating"].notna()) & (predicted_test_8["predicted_rating"] != -1)]

In [ ]:
predicted_test_9.head()

,user_id,anime_id,rating,predicted_rating
0,44017,13161,4,8.066667
1,14307,14993,7,8.000000
2,55155,268,9,9.142857
3,63515,2889,9,9.333333
4,54059,2581,7,6.000000


In [ ]:
mean_squared_error(predicted_test_9["rating"], predicted_test_9["predicted_rating"])

1.8150379611757528

In [ ]:
predicted_test_10 = predicted_test_8.copy()
predicted_test_10_greater_0 = predicted_test_8[(predicted_test_8["predicted_rating"].notna()) & (predicted_test_8["predicted_rating"] != -1)]
predicted_test_10_invalid = predicted_test_8[(predicted_test_8["predicted_rating"].isna()) | (predicted_test_8["predicted_rating"] == -1)]

In [ ]:
predicted_test_10_invalid_movie_mean = predicted_test_10_invalid.merge(avg_user_train_rating, how = "left", on = "user_id", suffixes = ("", "_train"))

<ipython-input-63-7d25a048799d>:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  predicted_test_10_invalid_movie_mean = predicted_test_10_invalid.merge(avg_user_train_rating, how = "left", on = "user_id", suffixes = ("", "_train"))
<ipython-input-63-7d25a048799d>:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  predicted_test_10_invalid_movie_mean = predicted_test_10_invalid.merge(avg_user_train_rating, how = "left", on = "user_id", suffixes = ("", "_train"))


In [ ]:
predicted_test_10_invalid_movie_mean["predicted_rating"] = predicted_test_10_invalid_movie_mean[("rating", "mean")]

In [ ]:
predicted_test_10_invalid_movie_mean = predicted_test_10_invalid_movie_mean[["user_id", "anime_id", "rating", "predicted_rating"]]

In [ ]:
predicted_test_10_invalid_movie_mean[(predicted_test_10_invalid_movie_mean["predicted_rating"].isna()) | (predicted_test_10_invalid_movie_mean["predicted_rating"] == -1)]

,user_id,anime_id,rating,predicted_rating
44,69461,269,10,NaN
106,70751,4722,9,NaN
119,1360,16498,10,NaN
282,45465,918,10,NaN
340,60842,853,9,NaN
...,...,...,...,...
105980,35,9001,10,NaN
106007,39943,1562,9,NaN
106057,2512,1535,9,NaN
106140,42660,9760,7,NaN


In [ ]:
predicted_test_10 = pd.concat([predicted_test_10_greater_0, predicted_test_10_invalid_movie_mean])

In [ ]:
predicted_test_10["predicted_rating"] = predicted_test_10["predicted_rating"].fillna(8.216)

In [ ]:
mean_squared_error(predicted_test_10["rating"], predicted_test_10["predicted_rating"])

1.8477696976907767

# Content-Based 100 neighbours & cosine^5 as weight (1.78)

In [ ]:
anime2 = anime.copy()

In [ ]:
anime2.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
anime2["text"] = anime2["name"] + " " + anime2["genre"]

In [ ]:
anime2["text"] = anime2["text"].astype(str)

In [ ]:
def remove_numbers_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: ''.join([i for i in text if not i.isdigit()])
        )
    return df

def remove_punctuations_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: re.sub(r'[^\w\s]', ' ', text)
        )
    df[dst_col] = df[src_col].apply(
        lambda text: text.replace("_", " ")
        )
    return df

def replace_multiple_spaces_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: re.sub(' +', ' ', text)
    )
    return df

def lowercase_string_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: text.lower()
        )
    return df

In [ ]:
anime2 = anime2.pipe(remove_numbers_df, src_col = "text", dst_col = "text").\
    pipe(remove_punctuations_df, src_col = "text", dst_col = "text").\
    pipe(replace_multiple_spaces_df, src_col = "text", dst_col = "text").\
    pipe(lowercase_string_df, src_col = "text", dst_col = "text")

In [ ]:
anime2.drop(["name", "genre"], axis = 1, inplace = True)

In [ ]:
anime2

,anime_id,type,episodes,rating,members,text
0,32281,Movie,1,9.37,200630,kimi no na wa drama romance school supernatural
1,5114,TV,64,9.26,793665,fullmetal alchemist brotherhood action adventu...
2,28977,TV,51,9.25,114262,gintama action comedy historical parody samura...
3,9253,TV,24,9.17,673572,steins gate sci fi thriller
4,9969,TV,51,9.16,151266,gintama action comedy historical parody samura...
...,...,...,...,...,...,...
12289,9316,OVA,1,4.15,211,toushindai my lover minami tai mecha minami he...
12290,5543,OVA,1,4.28,183,under world hentai
12291,5621,OVA,4,4.88,219,violence gekiga david no hoshi hentai
12292,6133,OVA,1,4.98,175,violence gekiga shin david no hoshi inma dense...


In [ ]:
vectorizer = TfidfVectorizer()
tfidf_text_vectors = vectorizer.fit_transform(anime2.text)

In [ ]:
tfidf_text_df = pd.DataFrame(tfidf_text_vectors.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
anime2.drop(["text"], axis = 1, inplace = True)

In [ ]:
anime2

,anime_id,type,episodes,rating,members
0,32281,Movie,1,9.37,200630
1,5114,TV,64,9.26,793665
2,28977,TV,51,9.25,114262
3,9253,TV,24,9.17,673572
4,9969,TV,51,9.16,151266
...,...,...,...,...,...
12289,9316,OVA,1,4.15,211
12290,5543,OVA,1,4.28,183
12291,5621,OVA,4,4.88,219
12292,6133,OVA,1,4.98,175


In [ ]:
mean_episodes = anime2[anime2.episodes != "Unknown"].episodes.astype(int).mean()

In [ ]:
anime2["episodes"] = anime2["episodes"].apply(lambda x: int(x) if x != "Unknown" else mean_episodes)

In [ ]:
scaler = MinMaxScaler()
scaler_vectors = scaler.fit_transform(anime2[["episodes", "rating", "members"]])

In [ ]:
scaler_df = pd.DataFrame(scaler_vectors, columns=scaler.get_feature_names_out())

In [ ]:
scaler_df

,episodes,rating,members
0,0.000000,0.924370,0.197872
1,0.034673,0.911164,0.782770
2,0.027518,0.909964,0.112689
3,0.012658,0.900360,0.664325
4,0.027518,0.899160,0.149186
...,...,...,...
12289,0.000000,0.297719,0.000203
12290,0.000000,0.313325,0.000176
12291,0.001651,0.385354,0.000211
12292,0.000000,0.397359,0.000168


In [ ]:
anime2 = pd.get_dummies(anime2)

In [ ]:
anime2

,anime_id,episodes,rating,members,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,32281,1.0,9.37,200630,1,0,0,0,0,0
1,5114,64.0,9.26,793665,0,0,0,0,0,1
2,28977,51.0,9.25,114262,0,0,0,0,0,1
3,9253,24.0,9.17,673572,0,0,0,0,0,1
4,9969,51.0,9.16,151266,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
12289,9316,1.0,4.15,211,0,0,0,1,0,0
12290,5543,1.0,4.28,183,0,0,0,1,0,0
12291,5621,4.0,4.88,219,0,0,0,1,0,0
12292,6133,1.0,4.98,175,0,0,0,1,0,0


In [ ]:
anime2.drop(["episodes", "rating", "members"], axis = 1, inplace = True)

In [ ]:
anime2 = pd.concat([anime2, scaler_df, tfidf_text_df], axis = 1)

In [ ]:
anime2.shape

(12294, 11862)

In [ ]:
anime2.rating = anime2.rating.fillna(anime2.rating.mean())

In [ ]:
anime2

,anime_id,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,episodes,rating,members,...,zushiki,zushioumaru,zusshiri,zutto,zvezda,zwei,zz,étude,üks,ēldlive
0,32281,1,0,0,0,0,0,0.000000,0.924370,0.197872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,0,0,0,0,0,1,0.034673,0.911164,0.782770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977,0,0,0,0,0,1,0.027518,0.909964,0.112689,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,0,0,0,0,0,1,0.012658,0.900360,0.664325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969,0,0,0,0,0,1,0.027518,0.899160,0.149186,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,9316,0,0,0,1,0,0,0.000000,0.297719,0.000203,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12290,5543,0,0,0,1,0,0,0.000000,0.313325,0.000176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12291,5621,0,0,0,1,0,0,0.001651,0.385354,0.000211,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12292,6133,0,0,0,1,0,0,0.000000,0.397359,0.000168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
nearest_neighbours = AnnoyIndex(anime2.shape[1] - 1, 'angular')
for index, row in anime2.drop(["anime_id"], axis = 1).iterrows():
    if index % 5000 == 0:
        print(index)
    nearest_neighbours.add_item(index, row)

nearest_neighbours.build(100)
# 4 mins

0
5000
10000


True

In [ ]:
neighbours = {}
num_neighbours = 100
for i in range(len(anime2)):
    if i % 5000 == 0:
        print(i)
    ith_neighbours = nearest_neighbours.get_nns_by_item(i, num_neighbours, include_distances = True)
    ith_id = anime2.loc[i, "anime_id"]

    temp = {}
    for j in range(len(ith_neighbours[0])):
        temp[anime.loc[ith_neighbours[0][j], "anime_id"]] = 1 - (ith_neighbours[1][j]) ** 2 / 2
        
    neighbours[ith_id] = temp

# 7 mins

0
5000
10000


In [ ]:
# train_dict = {}
# for index, row in train.iterrows():
#     temp_dict = train_dict.get(int(row["user_id"]), {})
#     temp_dict[int(row["anime_id"])] = int(row["rating"])
#     train_dict[int(row["user_id"])] = temp_dict

In [ ]:
predicted_test_8 = test.copy()
predicted_test_8.loc[:, "predicted_rating"] = -1

for index, row in predicted_test_8.iterrows():
    user_id = row["user_id"]
    anime_id = row["anime_id"]

    if anime_id not in neighbours:
        continue
    anime_id_neighbours = neighbours[anime_id]

    if user_id not in train_dict:
        continue
    user_ratings_train = train_dict[user_id]

    if index % 50000 == 0:
        print(index)

    sum = 0
    weights = 0
    for neighbour in anime_id_neighbours:
        if neighbour in user_ratings_train:
            sum += user_ratings_train[neighbour] * (anime_id_neighbours[neighbour]) ** 5
            weights += (anime_id_neighbours[neighbour]) ** 5

    if sum != 0:
        predicted_test_8.loc[index, "predicted_rating"] = sum / weights

# 6 mins

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000


In [ ]:
predicted_test_8[predicted_test_8["predicted_rating"] == -1]

,user_id,anime_id,rating,predicted_rating
11,41101,304,10,-1.0
71,69875,27655,10,-1.0
73,2063,18393,10,-1.0
86,44216,12445,9,-1.0
133,11269,6707,9,-1.0
...,...,...,...,...
1901043,9133,1078,6,-1.0
1901115,48526,15037,5,-1.0
1901121,73420,901,3,-1.0
1901130,62099,11371,9,-1.0


In [ ]:
predicted_test_9 = predicted_test_8[(predicted_test_8["predicted_rating"] != -1)]

In [ ]:
predicted_test_9.head()

,user_id,anime_id,rating,predicted_rating
0,44017,13161,4,8.079936
1,14307,14993,7,8.000000
2,55155,268,9,9.143428
3,63515,2889,9,9.221622
4,54059,2581,7,5.942792


In [ ]:
mean_squared_error(predicted_test_9["rating"], predicted_test_9["predicted_rating"])

1.7465676524740192

In [ ]:
predicted_test_10 = predicted_test_8.copy()
predicted_test_10_greater_0 = predicted_test_8[(predicted_test_8["predicted_rating"] != -1)]
predicted_test_10_invalid = predicted_test_8[(predicted_test_8["predicted_rating"] == -1)]

In [ ]:
predicted_test_10_invalid_movie_mean = predicted_test_10_invalid.merge(avg_user_train_rating, how = "left", on = "user_id", suffixes = ("", "_train"))

<ipython-input-267-7d25a048799d>:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  predicted_test_10_invalid_movie_mean = predicted_test_10_invalid.merge(avg_user_train_rating, how = "left", on = "user_id", suffixes = ("", "_train"))
<ipython-input-267-7d25a048799d>:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  predicted_test_10_invalid_movie_mean = predicted_test_10_invalid.merge(avg_user_train_rating, how = "left", on = "user_id", suffixes = ("", "_train"))


In [ ]:
predicted_test_10_invalid_movie_mean["predicted_rating"] = predicted_test_10_invalid_movie_mean[("rating", "mean")]

In [ ]:
predicted_test_10_invalid_movie_mean = predicted_test_10_invalid_movie_mean[["user_id", "anime_id", "rating", "predicted_rating"]]

In [ ]:
predicted_test_10_invalid_movie_mean[(predicted_test_10_invalid_movie_mean["predicted_rating"].isna()) | (predicted_test_10_invalid_movie_mean["predicted_rating"] == -1)]

,user_id,anime_id,rating,predicted_rating
44,69461,269,10,NaN
106,70751,4722,9,NaN
119,1360,16498,10,NaN
282,45465,918,10,NaN
340,60842,853,9,NaN
...,...,...,...,...
105980,35,9001,10,NaN
106007,39943,1562,9,NaN
106057,2512,1535,9,NaN
106140,42660,9760,7,NaN


In [ ]:
predicted_test_10 = pd.concat([predicted_test_10_greater_0, predicted_test_10_invalid_movie_mean])

In [ ]:
predicted_test_10["predicted_rating"] = predicted_test_10["predicted_rating"].fillna(8.216)

In [ ]:
mean_squared_error(predicted_test_10["rating"], predicted_test_10["predicted_rating"])

1.7831245177317778

# Content-Based 500 neighbours & cosine^9 as weight(1.67)

In [ ]:
anime2 = anime.copy()

In [ ]:
anime2.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
anime2["text"] = anime2["name"] + " " + anime2["genre"]

In [ ]:
anime2["text"] = anime2["text"].astype(str)

In [ ]:
def remove_numbers_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: ''.join([i for i in text if not i.isdigit()])
        )
    return df

def remove_punctuations_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: re.sub(r'[^\w\s]', ' ', text)
        )
    df[dst_col] = df[src_col].apply(
        lambda text: text.replace("_", " ")
        )
    return df

def replace_multiple_spaces_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: re.sub(' +', ' ', text)
    )
    return df

def lowercase_string_df(df, src_col, dst_col) -> pd.DataFrame:
    df[dst_col] = df[src_col].apply(
        lambda text: text.lower()
        )
    return df

In [ ]:
anime2 = anime2.pipe(remove_numbers_df, src_col = "text", dst_col = "text").\
    pipe(remove_punctuations_df, src_col = "text", dst_col = "text").\
    pipe(replace_multiple_spaces_df, src_col = "text", dst_col = "text").\
    pipe(lowercase_string_df, src_col = "text", dst_col = "text")

In [ ]:
anime2.drop(["name", "genre"], axis = 1, inplace = True)

In [ ]:
anime2

,anime_id,type,episodes,rating,members,text
0,32281,Movie,1,9.37,200630,kimi no na wa drama romance school supernatural
1,5114,TV,64,9.26,793665,fullmetal alchemist brotherhood action adventu...
2,28977,TV,51,9.25,114262,gintama action comedy historical parody samura...
3,9253,TV,24,9.17,673572,steins gate sci fi thriller
4,9969,TV,51,9.16,151266,gintama action comedy historical parody samura...
...,...,...,...,...,...,...
12289,9316,OVA,1,4.15,211,toushindai my lover minami tai mecha minami he...
12290,5543,OVA,1,4.28,183,under world hentai
12291,5621,OVA,4,4.88,219,violence gekiga david no hoshi hentai
12292,6133,OVA,1,4.98,175,violence gekiga shin david no hoshi inma dense...


In [ ]:
vectorizer = TfidfVectorizer()
tfidf_text_vectors = vectorizer.fit_transform(anime2.text)

In [ ]:
tfidf_text_df = pd.DataFrame(tfidf_text_vectors.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
anime2.drop(["text"], axis = 1, inplace = True)

In [ ]:
anime2

,anime_id,type,episodes,rating,members
0,32281,Movie,1,9.37,200630
1,5114,TV,64,9.26,793665
2,28977,TV,51,9.25,114262
3,9253,TV,24,9.17,673572
4,9969,TV,51,9.16,151266
...,...,...,...,...,...
12289,9316,OVA,1,4.15,211
12290,5543,OVA,1,4.28,183
12291,5621,OVA,4,4.88,219
12292,6133,OVA,1,4.98,175


In [ ]:
mean_episodes = anime2[anime2.episodes != "Unknown"].episodes.astype(int).mean()

In [ ]:
anime2["episodes"] = anime2["episodes"].apply(lambda x: int(x) if x != "Unknown" else mean_episodes)

In [ ]:
scaler = MinMaxScaler()
scaler_vectors = scaler.fit_transform(anime2[["episodes", "rating", "members"]])

In [ ]:
scaler_df = pd.DataFrame(scaler_vectors, columns=scaler.get_feature_names_out())

In [ ]:
scaler_df

,episodes,rating,members
0,0.000000,0.924370,0.197872
1,0.034673,0.911164,0.782770
2,0.027518,0.909964,0.112689
3,0.012658,0.900360,0.664325
4,0.027518,0.899160,0.149186
...,...,...,...
12289,0.000000,0.297719,0.000203
12290,0.000000,0.313325,0.000176
12291,0.001651,0.385354,0.000211
12292,0.000000,0.397359,0.000168


In [ ]:
anime2 = pd.get_dummies(anime2)

In [ ]:
anime2

,anime_id,episodes,rating,members,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,32281,1.0,9.37,200630,1,0,0,0,0,0
1,5114,64.0,9.26,793665,0,0,0,0,0,1
2,28977,51.0,9.25,114262,0,0,0,0,0,1
3,9253,24.0,9.17,673572,0,0,0,0,0,1
4,9969,51.0,9.16,151266,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
12289,9316,1.0,4.15,211,0,0,0,1,0,0
12290,5543,1.0,4.28,183,0,0,0,1,0,0
12291,5621,4.0,4.88,219,0,0,0,1,0,0
12292,6133,1.0,4.98,175,0,0,0,1,0,0


In [ ]:
anime2.drop(["episodes", "rating", "members"], axis = 1, inplace = True)

In [ ]:
anime2 = pd.concat([anime2, scaler_df, tfidf_text_df], axis = 1)

In [ ]:
anime2.shape

(12294, 11862)

In [ ]:
anime2.rating = anime2.rating.fillna(anime2.rating.mean())

In [ ]:
anime2

,anime_id,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,episodes,rating,members,...,zushiki,zushioumaru,zusshiri,zutto,zvezda,zwei,zz,étude,üks,ēldlive
0,32281,1,0,0,0,0,0,0.000000,0.924370,0.197872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,0,0,0,0,0,1,0.034673,0.911164,0.782770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977,0,0,0,0,0,1,0.027518,0.909964,0.112689,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,0,0,0,0,0,1,0.012658,0.900360,0.664325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969,0,0,0,0,0,1,0.027518,0.899160,0.149186,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,9316,0,0,0,1,0,0,0.000000,0.297719,0.000203,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12290,5543,0,0,0,1,0,0,0.000000,0.313325,0.000176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12291,5621,0,0,0,1,0,0,0.001651,0.385354,0.000211,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12292,6133,0,0,0,1,0,0,0.000000,0.397359,0.000168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
nearest_neighbours = AnnoyIndex(anime2.shape[1] - 1, 'angular')
for index, row in anime2.drop(["anime_id"], axis = 1).iterrows():
    if index % 5000 == 0:
        print(index)
    nearest_neighbours.add_item(index, row)

nearest_neighbours.build(100)
# 5 mins

0
5000
10000


True

In [ ]:
neighbours = {}
num_neighbours = 500
for i in range(len(anime2)):
    if i % 5000 == 0:
        print(i)
    ith_neighbours = nearest_neighbours.get_nns_by_item(i, num_neighbours, include_distances = True)
    ith_id = anime2.loc[i, "anime_id"]

    temp = {}
    for j in range(len(ith_neighbours[0])):
        temp[anime.loc[ith_neighbours[0][j], "anime_id"]] = 1 - (ith_neighbours[1][j]) ** 2 / 2
        
    neighbours[ith_id] = temp

# 13 mins

0
5000
10000


In [ ]:
train_dict = {}
for index, row in train.iterrows():
    temp_dict = train_dict.get(int(row["user_id"]), {})
    temp_dict[int(row["anime_id"])] = int(row["rating"])
    train_dict[int(row["user_id"])] = temp_dict

In [ ]:
predicted_test_8 = test.copy()
predicted_test_8.loc[:, "predicted_rating"] = -1

for index, row in predicted_test_8.iterrows():
    user_id = row["user_id"]
    anime_id = row["anime_id"]

    if anime_id not in neighbours:
        continue
    anime_id_neighbours = neighbours[anime_id]

    if user_id not in train_dict:
        continue
    user_ratings_train = train_dict[user_id]

    if index % 50000 == 0:
        print(index)

    sum = 0
    weights = 0
    for neighbour in anime_id_neighbours:
        if neighbour in user_ratings_train:
            sum += user_ratings_train[neighbour] * (anime_id_neighbours[neighbour]) ** 9
            weights += (anime_id_neighbours[neighbour]) ** 9

    if sum != 0:
        predicted_test_8.loc[index, "predicted_rating"] = sum / weights

# 6 mins

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000


In [ ]:
predicted_test_8[predicted_test_8["predicted_rating"] == -1]

,user_id,anime_id,rating,predicted_rating
73,2063,18393,10,-1.0
145,45884,15687,8,-1.0
210,9756,528,10,-1.0
235,54306,114,8,-1.0
285,10945,2448,10,-1.0
...,...,...,...,...
1900995,67109,7338,7,-1.0
1901000,39939,10067,7,-1.0
1901043,9133,1078,6,-1.0
1901115,48526,15037,5,-1.0


In [ ]:
predicted_test_9 = predicted_test_8[(predicted_test_8["predicted_rating"] != -1)]

In [ ]:
predicted_test_9.head()

,user_id,anime_id,rating,predicted_rating
0,44017,13161,4,7.542911
1,14307,14993,7,7.147282
2,55155,268,9,9.071852
3,63515,2889,9,9.152393
4,54059,2581,7,7.471401


In [ ]:
mean_squared_error(predicted_test_9["rating"], predicted_test_9["predicted_rating"])

1.6547882612040963

In [ ]:
predicted_test_10 = predicted_test_8.copy()
predicted_test_10_greater_0 = predicted_test_8[(predicted_test_8["predicted_rating"] != -1)]
predicted_test_10_invalid = predicted_test_8[(predicted_test_8["predicted_rating"] == -1)]

In [ ]:
predicted_test_10_invalid_movie_mean = predicted_test_10_invalid.merge(avg_user_train_rating, how = "left", on = "user_id", suffixes = ("", "_train"))

<ipython-input-89-7d25a048799d>:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  predicted_test_10_invalid_movie_mean = predicted_test_10_invalid.merge(avg_user_train_rating, how = "left", on = "user_id", suffixes = ("", "_train"))
<ipython-input-89-7d25a048799d>:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  predicted_test_10_invalid_movie_mean = predicted_test_10_invalid.merge(avg_user_train_rating, how = "left", on = "user_id", suffixes = ("", "_train"))


In [ ]:
predicted_test_10_invalid_movie_mean["predicted_rating"] = predicted_test_10_invalid_movie_mean[("rating", "mean")]

In [ ]:
predicted_test_10_invalid_movie_mean = predicted_test_10_invalid_movie_mean[["user_id", "anime_id", "rating", "predicted_rating"]]

In [ ]:
predicted_test_10_invalid_movie_mean[(predicted_test_10_invalid_movie_mean["predicted_rating"].isna()) | (predicted_test_10_invalid_movie_mean["predicted_rating"] == -1)]

,user_id,anime_id,rating,predicted_rating
14,69461,269,10,NaN
32,70751,4722,9,NaN
36,1360,16498,10,NaN
87,45465,918,10,NaN
101,60842,853,9,NaN
...,...,...,...,...
27903,35,9001,10,NaN
27910,39943,1562,9,NaN
27923,2512,1535,9,NaN
27940,42660,9760,7,NaN


In [ ]:
predicted_test_10 = pd.concat([predicted_test_10_greater_0, predicted_test_10_invalid_movie_mean])

In [ ]:
predicted_test_10["predicted_rating"] = predicted_test_10["predicted_rating"].fillna(8.216)

In [ ]:
mean_squared_error(predicted_test_10["rating"], predicted_test_10["predicted_rating"])

1.6688186692302585